In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
file_path = "https://raw.githubusercontent.com/VieEeEw/CS296-project1/master/data.csv"

In [4]:
rawDF = pd.read_csv(file_path, low_memory=False)
rawDF = rawDF.fillna(0)[['Fall', 'College', 'Major Name', 'Total']]
rawDF['Total'] = pd.to_numeric(rawDF['Total']
                               .astype(str)
                               .replace(",","", regex=True)
                               .loc[rawDF['Total'] != 'Art and Design'])

procDF = (rawDF
          .loc[rawDF['Major Name'].str.isupper() == False]
         .groupby(['Fall', 'College', 'Major Name'], as_index=False)
          .sum())

outDF = (procDF
         .set_index(["Fall", "College", 'Major Name'])
         .unstack(level=0)
         .fillna(0)
         .stack()
         .reset_index()
        )


test = outDF.loc[outDF['College'] == "Engineering"][['Major Name', 'Fall', 'Total']]

totals = outDF.groupby(['College', 'Fall'])['Total'].sum().reset_index()

totals['Major Name'] = 'Total'

finalDF = pd.concat([outDF, totals], sort=False).sort_values(['College','Major Name', 'Fall', 'Total'])


json_out = (finalDF.groupby(['College', 'Major Name'])
    .apply(lambda x: x[['Fall', 'Total']].to_dict('records'))
    .reset_index()
    .rename(columns={0:'Count'})
    .groupby(['College'])
    .apply(lambda x: dict(zip(x['Major Name'], x['Count'])))
    .reset_index()
    .rename(columns={0:'Major'})
    .to_json(orient='records')
)


# json.loads(json_out)

with open('out.json', 'w') as file:
  json.dump(json_out, file)
  